<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# Instrumenting AWS Bedrock client with OpenInference and Phoenix

In this tutorial we will trace model calls to AWS Bedrock using OpenInference. The OpenInference Bedrock tracer instruments the Python `boto3` library, so all `invoke_model` calls will automatically generate traces that can be sent to Phoenix.

ℹ️ This notebook requires a valid AWS configuration and access to AWS Bedrock and the `claude-v2` model from Anthropic.

## 1. Install dependencies and set up OpenTelemetry tracer

First install dependencies

In [ ]:
!pip install arize-phoenix boto3 openinference-instrumentation-bedrock

Import libraries

In [ ]:
import json
from urllib.parse import urljoin

import boto3
from openinference.instrumentation.bedrock import BedrockInstrumentor
from opentelemetry.sdk.trace.export import ConsoleSpanExporter

import phoenix as px
from phoenix.otel import SimpleSpanProcessor, register

Start a Pheonix server to collect traces. Be sure to view Phoenix in your browser to watch traces show up in Phoenix as they are collected.

In [ ]:
px.launch_app().view()
session_url = px.active_session().url

Here we're configuring the OpenTelemetry tracer by adding two SpanProcessors. The first SpanProcessor will simply print all traces received from OpenInference instrumentation to the console. The second will export traces to Phoenix so they can be collected and viewed.

In [ ]:
phoenix_otlp_endpoint = urljoin(session_url, "v1/traces")
tracer_provider = register()
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=ConsoleSpanExporter()))
tracer_provider.add_span_processor(SimpleSpanProcessor(endpoint=phoenix_otlp_endpoint))

## 2. Instrumenting Bedrock clients

Now, let's create a `boto3` session. This initiates a configured environment for interacting with AWS services. If you haven't yet configured `boto3` to use your credentials, please refer to the [official documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html). Or, if you have the AWS CLI, run `aws configure` from your terminal.

In [ ]:
session = boto3.session.Session()

Clients created using this session configuration are currently uninstrumented. We'll make one for comparison.

In [ ]:
uninstrumented_client = session.client("bedrock-runtime")

Now we instrument Bedrock with our OpenInference instrumentor. All Bedrock clients created after this call will automatically produce traces when calling `invoke_model`.

In [ ]:
BedrockInstrumentor().instrument(skip_dep_check=True)
instrumented_client = session.client("bedrock-runtime")

## 3. Calling the LLM and viewing OpenInference traces

Calling `invoke_model` using the `uninstrumented_client` will produce no traces, but will show the output from the LLM.

In [ ]:
prompt = b'{"prompt": "Human: Hello there, how are you? Assistant:", "max_tokens_to_sample": 1024}'
response = uninstrumented_client.invoke_model(modelId="anthropic.claude-v2", body=prompt)
response_body = json.loads(response.get("body").read())
print(response_body["completion"])

LLM calls using the `instrumented_client` will print traces to the console! By configuring the `SpanProcessor` to export to a different OpenTelemetry collector, your OpenInference spans can be collected and analyzed to better understand the behavior of your LLM application.

In [ ]:
response = instrumented_client.invoke_model(modelId="anthropic.claude-v2", body=prompt)
response_body = json.loads(response.get("body").read())
print(response_body["completion"])

More information about our instrumentation integrations, OpenInerence can be found in our [documentation](https://docs.arize.com/phoenix/telemetry/instrumentation)